In [209]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import glob
import time
import yfinance as yf
import datetime
import requests


# options = Options()
# options.add_argument("--start-maximized")  # Maximize the browser window
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-popup-blocking")
# options.add_argument("--disable-infobars")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-gpu")
# options.add_argument("--ignore-certificate-errors")

#driver = webdriver.Chrome()
#driver = webdriver.Safari()

ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]")),
                'tipranks.com': EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

buysell_tipranks_dict = {'Strong Buy': '1', 'Moderate Buy': '2', 'Hold': '3', }

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 5).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
        try:
            accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
            accept.click()
        except:
            print("No 'Accept' button found or unable to click on ", site)
    
    elif site == 'tipranks.com':
        try:
            accept_button = WebDriverWait(driver, 5).until(cookies_ec_dict['tipranks.com'])
            accept_button.click()
            print("Accept button clicked successfully")
        except Exception as e:
            print("Error:", e)

    elif site == 'stockinvest.us':
        pass

    elif site == 'tradingview.com':
        pass

    elif site == 'investing.com':
        accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
        accept.click()



def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    time.sleep(1)

    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    time.sleep(0.5)

    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True

    elif website == 'tipranks.com':
        try:
            rank_span = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".colorpale.fonth4_bold.aligncenter"))
            )

            # Get the text of the span element
            rank_text = rank_span.text.strip()
            return rank_text
        except Exception as e:
            ERROR = True

    elif website == 'stockinvest.us':
        try:
            technicals_rank = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, "//span[@class='pl-5 pr-5 pt-3 pb-3 font-weight-600 border-radius-default font-size-16']"))
            )

            # Get the text of the element
            technicals_rank = number_element.text.strip()
            print(technicals_rank)
            return technicals_rank      
        except Exception as e:
            print('falha')
            ERROR = True


    elif website == 'tradingview.com':
        try:
            speedometer_div = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".speedometerWrapper-kg4MJrFB.summary-kg4MJrFB"))
            )
            # Get the outer HTML content of the div element
            html_content = speedometer_div.get_attribute("outerHTML")
            soup = BeautifulSoup(html_content, 'html.parser')
            counter_spans = soup.find_all('span', class_='counterNumber-kg4MJrFB')
            numbers = [int(span.text) for span in counter_spans]
            if numbers is not None and sum(numbers)!=0:
                return (-numbers[0]+numbers[-1])/sum(numbers)
        except Exception as e:
            print('falha')
            ERROR = True


            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    elif website == 'tipranks.com':
        driver.get(f'https://www.tipranks.com/stocks/{symbol}/forecast')
    elif website == 'stockinvest.us':
        driver.get(f'https://stockinvest.us/stock/{symbol}')
    elif website == 'tradingview.com':
        driver.get(f'https://www.tradingview.com/symbols/{symbol}/technicals')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    #google_site('earnings calendar', 'investing.com', cookies = True)
    driver.get('https://www.investing.com/earnings-calendar/')
    time.sleep(0.5)
    accept_cookies('investing.com')
    time.sleep(0.5)
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(openclose[index])

    
         
def filter_by_lastrank(path, year, month, day, website):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        ranks = ['1','2','3','4','5','None']
        ticker_rank = {rank:[] for rank in ranks}
        for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            with open(tickerpath+website, 'r') as f:
                text = f.read()
                last = text.split(' ')[-1]
                ticker_rank[last[:last.find('\n')]].append(ticker)

        return ticker_rank
    

def get_tickers_on_trading212():
    url = "https://live.trading212.com/api/v0/equity/metadata/instruments"

    headers = {"Authorization": f"{open('.apikey', 'r').read()[:-1]}"}

    response = requests.get(url, headers=headers)

    data = response.json()

    trading212_tickers = []
    for instrument in data:
        trading212_tickers.append(instrument['shortName']) 

    return trading212_tickers

##########################################


In [210]:
#trading212_tickers = get_tickers_on_trading212()

In [211]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 11
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
driver.quit()

In [212]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)

ORCL none
LEGN open
CASY close
MTN none
AGRPY none
DLO none
NTCO close
ASAN none
FTRE open
ETV none
OCUL none
GRCL none
BLDP open
ETW none
YALA close
AVO none
NOAH none
CVGW none
GNE open
BWMN none
ZH none
EXK open
ETB none
CDMO none
PACK none
NLCP open
AVD none
LFMD none
MYPS none
FANH none
DM none
OVID none
NNBR none
CANG none
HRTG none
FREE none
CLLS none
CTMX none
ALTO none
MRCC none
AXR none
TKNO none
VOC none
BRID none
ELTK open
ASRT none
SND none
ENZ none
CYTH none
FTEK none
BTCS none
OCC none
FENG none
ROYTL none
SONX none
VRME none
ONVC none
FXLV none
CANOQ none
PUDA none


In [213]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [214]:
year, month, day = 2024, 3, 11
driver = webdriver.Chrome()
earnings_rank('tipranks.com', year, month, day, path = 'earnings')
driver.quit()

Accept button clicked successfully
CTMX None
CANOQ None


In [180]:
year, month, day = 2024, 3, 7
ticker_rank = filter_by_lastrank('earnings', year, month, day, 'zacks.com')
up_to_ranking = 2


In [181]:
total_number_of_companies = 0
for i in range(1,up_to_ranking + 1):
    print(f'{i}: ', ticker_rank[f'{i}'], '\n')
    total_number_of_companies += len(ticker_rank[f'{i}'])

print('total number of companies: ', total_number_of_companies)

1:  ['GPS', 'DOCU', 'AEO'] 

2:  ['CGEM', 'DLAKY', 'DESP', 'PSNY', 'REAX', 'OCUL', 'COST', 'CTTAY', 'SVV', 'VITL', 'TOI', 'PROF', 'EYPT', 'AVVIY', 'PBPB', 'ALDX', 'MRVL'] 

total number of companies:  20


In [182]:

total_number_of_companies = 0
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank[f'{i}'] if x in trading212_tickers]
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

print('total number of companies: ', total_number_of_companies)

1:  ['GPS', 'DOCU', 'AEO'] 

2:  ['CGEM', 'DLAKY', 'PSNY', 'OCUL', 'COST', 'SVV', 'VITL', 'TOI', 'PROF', 'EYPT', 'ALDX', 'MRVL'] 

total number of companies:  15


In [192]:
ticker_rank = filter_by_lastrank('earnings', year, month, day, 'zacks.com')
initial_value_per_ticker = 1
total_initial_value = 0
total_final_value = 0
total_percentage = 0
start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 10))
end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 1))
for i in range(1,up_to_ranking + 1):
    for tic in ticker_rank[f'{i}']:
        if tic in trading212_tickers and tic != 'EVA':
            yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
            open_values = yf_ticker['Open'].values
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
                  {np.round(initial_value_per_ticker*percentage_difference,2)},    \
                  {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

                        Open   High      Low  Close  Adj Close  Volume
Datetime                                                              
2024-03-07 04:00:00  19.1000  19.10  19.1000  19.10      19.10       0
2024-03-07 04:02:00  19.2100  19.21  19.2100  19.21      19.21       0
2024-03-07 04:07:00  19.2000  19.20  19.2000  19.20      19.20       0
2024-03-07 04:13:00  19.2000  19.20  19.2000  19.20      19.20       0
2024-03-07 04:26:00  19.2100  19.22  19.2100  19.22      19.22       0
...                      ...    ...      ...    ...        ...     ...
2024-03-07 19:55:00  20.3700  20.37  20.3500  20.37      20.37       0
2024-03-07 19:56:00  20.3691  20.37  20.3400  20.34      20.34       0
2024-03-07 19:57:00  20.3300  20.33  20.3200  20.32      20.32       0
2024-03-07 19:58:00  20.3300  20.36  20.3203  20.36      20.36       0
2024-03-07 19:59:00  20.3700  20.45  20.3700  20.42      20.42       0

[691 rows x 6 columns]
GPS, 20.24,   20.37,                    0.64,        

In [62]:
yf_ticker

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-02-26,56.470001,56.630001,55.820000,56.189999,56.189999,1905300
2024-02-27,56.279999,56.480000,56.090000,56.369999,56.369999,1767200
2024-02-28,56.310001,56.639999,56.200001,56.320000,56.320000,1920200
2024-02-29,56.669998,56.680000,56.080002,56.200001,56.200001,4017200
2024-03-01,56.290001,56.680000,55.520000,56.529999,56.529999,3524700
2024-03-04,56.520000,57.130001,56.299999,56.970001,56.970001,3621800
2024-03-05,57.000000,58.459999,56.900002,56.930000,56.930000,3504600
2024-03-06,57.130001,57.930000,56.750000,57.889999,57.889999,3833000


In [143]:
yf_ticker = yf.download(tickers='SPY', start='2024-03-01', end ='2024-03-06', progress=False, ignore_tz=True)


In [144]:
yf_ticker

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-01,508.980011,513.289978,508.559998,512.849976,512.849976,76805900
2024-03-04,512.030029,514.200012,512.000000,512.299988,512.299988,49799300
2024-03-05,510.239990,510.700012,504.910004,507.179993,507.179993,72670700


In [145]:
yf_ticker['Open'].values

array([508.98001099, 512.0300293 , 510.23999023])